<a href="https://colab.research.google.com/github/currencyfxjle/Rep-Customer-Item-sales-by-Week-Month-invoices-credits-/blob/main/Rep_Customer_Item_sales_by_Week_%26_Month_(invoices_credits).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WEEKLY

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
            pivot = weekly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)
        matrix_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

MONTHLY

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Monthly sales matrices calculations
        def calculate_matrix(groupby_column):
            monthly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='M')]).agg({'Amount': 'sum'}).reset_index()
            pivot = monthly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='M'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)
        matrix_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file (only works in Google Colab)
from google.colab import files
files.download('output_file.xlsx')

**INVOICES & CREDITS BY WEEK**

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Define a function to calculate matrices for invoices and credit memos separately
    def calculate_matrix(groupby_column):
        # Read the data for the current sheet
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing (same as before)
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Separate invoices and credit memos
        invoices = df[df['Type'] == 'Invoice']
        credit_memos = df[df['Type'] == 'Credit Memo']

        # Calculate matrices for invoices and credit memos
        weekly_sales_invoices = invoices.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
        weekly_sales_credit_memos = credit_memos.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()

        # Pivot tables for invoices and credit memos
        pivot_invoices = weekly_sales_invoices.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))
        pivot_credit_memos = weekly_sales_credit_memos.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

        # Insert totals and concatenate invoices and credit memos
        pivot_invoices.loc['TOTALS', :] = pivot_invoices.sum()
        pivot_credit_memos.loc['TOTALS', :] = pivot_credit_memos.sum()

        combined_matrix = pd.concat([pivot_invoices, pivot_credit_memos], axis=0, keys=['Invoices', 'Credit Memos'])

        return combined_matrix

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        matrix_combined_df = calculate_matrix('Rep')

        # Save the combined matrix for the current sheet to Excel
        matrix_combined_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)

        # Autofit columns for the matrix sheet
        worksheet = writer.sheets[f'{sheet_name} Rep']
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

        # Now, apply the same logic to 'Item' and 'Name' tabs
        matrix_combined_item_df = calculate_matrix('Item')
        matrix_combined_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)

        matrix_combined_name_df = calculate_matrix('Name')
        matrix_combined_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

**INVOICES & CREDITS BY MONTH**

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Define a function to calculate matrices for invoices and credit memos separately
    def calculate_matrix(groupby_column):
        # Read the data for the current sheet
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing (same as before)
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Separate invoices and credit memos
        invoices = df[df['Type'] == 'Invoice']
        credit_memos = df[df['Type'] == 'Credit Memo']

        # Calculate matrices for invoices and credit memos
        monthly_sales_invoices = invoices.groupby([groupby_column, pd.Grouper(key='Date', freq='M')]).agg({'Amount': 'sum'}).reset_index()
        monthly_sales_credit_memos = credit_memos.groupby([groupby_column, pd.Grouper(key='Date', freq='M')]).agg({'Amount': 'sum'}).reset_index()

        # Pivot tables for invoices and credit memos
        pivot_invoices = monthly_sales_invoices.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='M'))
        pivot_credit_memos = monthly_sales_credit_memos.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='M'))

        # Insert totals and concatenate invoices and credit memos
        pivot_invoices.loc['TOTALS', :] = pivot_invoices.sum()
        pivot_credit_memos.loc['TOTALS', :] = pivot_credit_memos.sum()

        combined_matrix = pd.concat([pivot_invoices, pivot_credit_memos], axis=0, keys=['Invoices', 'Credit Memos'])

        return combined_matrix

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        matrix_combined_df = calculate_matrix('Rep')

        # Save the combined matrix for the current sheet to Excel
        matrix_combined_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)

        # Autofit columns for the matrix sheet
        worksheet = writer.sheets[f'{sheet_name} Rep']
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

        # Now, apply the same logic to 'Item' and 'Name' tabs
        matrix_combined_item_df = calculate_matrix('Item')
        matrix_combined_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)

        matrix_combined_name_df = calculate_matrix('Name')
        matrix_combined_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

# Download the Excel file (only works in Google Colab)
from google.colab import files
files.download('output_file.xlsx')